In [19]:
# Data preparation
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Load data
Sales_df = pd.read_csv("preprocessed_data/sales.csv", index_col=0)
Ingredients_df = pd.read_csv("preprocessed_data/ingredients.csv", index_col=0)

In [20]:
Sales_df.head()

,pizza_id,order_id,pizza_name_id,quantity,order_date,order_time,unit_price,total_price,pizza_size,pizza_category,pizza_ingredients,pizza_name
0,1,1,hawaiian_m,1,2015-01-01 00:00:00,11:38:36,13.25,13.25,M,Classic,"Sliced Ham, Pineapple, Mozzarella Cheese",The Hawaiian Pizza
1,2,2,classic_dlx_m,1,2015-01-01 00:00:00,11:57:40,16.00,16.00,M,Classic,"Pepperoni, Mushrooms, Red Onions, Red Peppers,...",The Classic Deluxe Pizza
2,3,2,five_cheese_l,1,2015-01-01 00:00:00,11:57:40,18.50,18.50,L,Veggie,"Mozzarella Cheese, Provolone Cheese, Smoked Go...",The Five Cheese Pizza
3,4,2,ital_supr_l,1,2015-01-01 00:00:00,11:57:40,20.75,20.75,L,Supreme,"Calabrese Salami, Capocollo, Tomatoes, Red Oni...",The Italian Supreme Pizza
4,5,2,mexicana_m,1,2015-01-01 00:00:00,11:57:40,16.00,16.00,M,Veggie,"Tomatoes, Red Peppers, Jalapeno Peppers, Red O...",The Mexicana Pizza


In [21]:
Sales_df.isnull().sum()

pizza_id             0
order_id             0
pizza_name_id        0
quantity             0
order_date           0
order_time           0
unit_price           0
total_price          0
pizza_size           0
pizza_category       0
pizza_ingredients    0
pizza_name           0
dtype: int64

In [22]:
Sales_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 48620 entries, 0 to 48619
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   pizza_id           48620 non-null  int64  
 1   order_id           48620 non-null  int64  
 2   pizza_name_id      48620 non-null  object 
 3   quantity           48620 non-null  int64  
 4   order_date         48620 non-null  object 
 5   order_time         48620 non-null  object 
 6   unit_price         48620 non-null  float64
 7   total_price        48620 non-null  float64
 8   pizza_size         48620 non-null  object 
 9   pizza_category     48620 non-null  object 
 10  pizza_ingredients  48620 non-null  object 
 11  pizza_name         48620 non-null  object 
dtypes: float64(2), int64(3), object(7)
memory usage: 4.8+ MB


In [23]:
Ingredients_df.isnull().sum()

pizza_name_id         0
pizza_name            0
pizza_ingredients     0
Items_Qty_In_Grams    0
dtype: int64

In [24]:
# Parse dates function with error logging
def parse_dates(date):
    for fmt in ('%d-%m-%Y', '%d/%m/%Y', '%Y-%m-%d %H:%M:%S'):
        try:
            return pd.to_datetime(date, format=fmt)
        except ValueError:
            pass
    raise ValueError(f'No valid date format found for {date}')

# Apply the function to the 'order_date' column
Sales_df['order_date'] = Sales_df['order_date'].apply(parse_dates)

In [25]:
# Prepare data for weekly aggregation
Sales_df = Sales_df[['order_date', 'pizza_name', 'quantity']]
sales_summary = Sales_df.groupby(['order_date', 'pizza_name']).sum().reset_index()

# Pivot the data to have pizza names as columns
sales_pivot = sales_summary.pivot(index='order_date', columns='pizza_name', values='quantity').fillna(0)

In [26]:
sales_pivot.head()

pizza_name,The Barbecue Chicken Pizza,The Big Meat Pizza,The Brie Carre Pizza,The Calabrese Pizza,The California Chicken Pizza,The Chicken Alfredo Pizza,The Chicken Pesto Pizza,The Classic Deluxe Pizza,The Five Cheese Pizza,The Four Cheese Pizza,...,The Prosciutto and Arugula Pizza,The Sicilian Pizza,The Soppressata Pizza,The Southwest Chicken Pizza,The Spicy Italian Pizza,The Spinach Pesto Pizza,The Spinach Supreme Pizza,The Spinach and Feta Pizza,The Thai Chicken Pizza,The Vegetables + Vegetables Pizza
order_date,,,,,,,,,,,,,,,,,,,,,
2015-01-01,11.0,5.0,0.0,1.0,4.0,2.0,4.0,10.0,7.0,6.0,...,4.0,4.0,1.0,7.0,5.0,2.0,5.0,2.0,8.0,3.0
2015-01-02,12.0,9.0,3.0,7.0,15.0,3.0,2.0,13.0,5.0,6.0,...,6.0,5.0,3.0,9.0,9.0,5.0,3.0,4.0,8.0,1.0
2015-01-03,2.0,3.0,1.0,1.0,6.0,0.0,1.0,4.0,5.0,4.0,...,1.0,2.0,0.0,9.0,4.0,3.0,0.0,5.0,4.0,6.0
2015-01-04,2.0,3.0,0.0,2.0,3.0,3.0,2.0,8.0,7.0,6.0,...,4.0,8.0,6.0,10.0,4.0,4.0,6.0,1.0,3.0,5.0
2015-01-05,6.0,1.0,1.0,4.0,11.0,2.0,5.0,9.0,6.0,4.0,...,5.0,5.0,0.0,6.0,5.0,2.0,4.0,5.0,10.0,6.0


In [27]:
sales_summary.head()

,order_date,pizza_name,quantity
0,2015-01-01,The Barbecue Chicken Pizza,11
1,2015-01-01,The Big Meat Pizza,5
2,2015-01-01,The Calabrese Pizza,1
3,2015-01-01,The California Chicken Pizza,4
4,2015-01-01,The Chicken Alfredo Pizza,2


In [28]:
Sales_df.tail()

,order_date,pizza_name,quantity
48615,2015-12-31,The Chicken Alfredo Pizza,1
48616,2015-12-31,The Four Cheese Pizza,1
48617,2015-12-31,The Napolitana Pizza,1
48618,2015-12-31,The Mexicana Pizza,1
48619,2015-12-31,The Barbecue Chicken Pizza,1


In [29]:
# Applying SARIMA Model for Forecasting

# Initialize a dictionary to store SARIMA models for each pizza
sarima_models = {}

# Fit the SARIMA model for each pizza type
for pizza_name in sales_pivot.columns:
    try:
        model = SARIMAX(sales_pivot[pizza_name], order=(1, 1, 0), seasonal_order=(1, 1, 0, 7))
        model_fit = model.fit(disp=False)
        sarima_models[pizza_name] = model_fit
    except Exception as e:
        print(f"SARIMA model for {pizza_name} failed to fit.")

c:\Users\DELL\anaconda3\envs\py8\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\DELL\anaconda3\envs\py8\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\DELL\anaconda3\envs\py8\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\DELL\anaconda3\envs\py8\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. f

In [30]:
# Generate Sales Forecast for Next Week (7 days)

prediction_days = 7

predictions_sarima = {}

# Generate forecast for the next 7 days for each pizza type
for pizza_name, model in sarima_models.items():
    predictions_sarima[pizza_name] = model.predict(start=len(sales_pivot), end=len(sales_pivot) + prediction_days - 1)

# Convert the predictions into a DataFrame
predictions_df = pd.DataFrame(predictions_sarima)

# Generate predicted dates
predicted_dates = pd.date_range(start=sales_pivot.index[-1] + pd.Timedelta(days=1), periods=prediction_days, freq='D')

# Assign predicted dates to predictions DataFrame
predictions_df.index = predicted_dates

c:\Users\DELL\anaconda3\envs\py8\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\DELL\anaconda3\envs\py8\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
c:\Users\DELL\anaconda3\envs\py8\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\DELL\anaconda3\envs\py8\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_predict

In [31]:
predictions_df.head()

,The Barbecue Chicken Pizza,The Big Meat Pizza,The Brie Carre Pizza,The Calabrese Pizza,The California Chicken Pizza,The Chicken Alfredo Pizza,The Chicken Pesto Pizza,The Classic Deluxe Pizza,The Five Cheese Pizza,The Four Cheese Pizza,...,The Prosciutto and Arugula Pizza,The Sicilian Pizza,The Soppressata Pizza,The Southwest Chicken Pizza,The Spicy Italian Pizza,The Spinach Pesto Pizza,The Spinach Supreme Pizza,The Spinach and Feta Pizza,The Thai Chicken Pizza,The Vegetables + Vegetables Pizza
2016-01-01,8.160714,3.027896,0.191648,2.268178,6.507168,2.194045,3.389738,7.393368,3.948329,7.727451,...,4.544626,0.634973,1.745454,4.797908,2.419269,4.851607,2.553220,3.056631,5.351353,7.437561
2016-01-02,7.773062,7.377793,0.865479,3.374618,6.495245,3.052564,4.267490,7.561603,3.641112,11.749415,...,5.289952,0.803174,2.991464,4.849589,5.233762,5.148542,3.993360,2.522785,8.236196,7.408238
2016-01-03,7.283069,4.185966,1.005429,2.077624,6.494081,1.072046,2.997033,7.445395,3.195455,11.074673,...,2.813993,2.336787,2.004835,3.320830,5.127789,5.581758,1.529954,1.951653,6.408649,10.577717
2016-01-04,9.611882,3.542319,0.125743,1.223214,6.504170,2.943628,4.886369,8.677715,2.043000,7.688107,...,5.802684,-0.927236,2.840617,3.355027,2.500047,3.522347,3.018285,4.209406,3.745625,7.307964
2016-01-05,8.617341,5.322685,0.559086,3.652895,6.508639,2.639193,1.682049,6.206635,2.455170,8.659632,...,3.670766,-0.218570,2.980137,4.308192,3.009983,4.824944,1.017486,1.594424,4.836520,5.279084


In [32]:
ingredients_df = Ingredients_df[['pizza_name', 'pizza_ingredients', 'Items_Qty_In_Grams']]

# change column name to items_qty
ingredients_df.rename(columns={'Items_Qty_In_Grams': 'items_qty'}, inplace=True)
ingredients_df

C:\Users\DELL\AppData\Local\Temp\ipykernel_20268\2376984744.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ingredients_df.rename(columns={'Items_Qty_In_Grams': 'items_qty'}, inplace=True)


,pizza_name,pizza_ingredients,items_qty
0,The Barbecue Chicken Pizza,Barbecued Chicken,40.0
1,The Barbecue Chicken Pizza,Red Peppers,15.0
2,The Barbecue Chicken Pizza,Green Peppers,20.0
3,The Barbecue Chicken Pizza,Tomatoes,30.0
4,The Barbecue Chicken Pizza,Red Onions,60.0
...,...,...,...
513,The Vegetables + Vegetables Pizza,Green Peppers,20.0
514,The Vegetables + Vegetables Pizza,Red Onions,20.0
515,The Vegetables + Vegetables Pizza,Zucchini,10.0
516,The Vegetables + Vegetables Pizza,Spinach,15.0


In [33]:
# Create a dictionary to store ingredient quantities
ingredient_quantities = {}

# Get the ingredients for the pizza
for pizza_name in predictions_df.columns:
    predicted_quantity = predictions_df[pizza_name].sum()
    pizza_ingredients = ingredients_df[ingredients_df['pizza_name'] == pizza_name]
    
    for index, row in pizza_ingredients.iterrows():
        ingredient = row['pizza_ingredients']
        ingredient_qty = row['items_qty']
        required_quantity = predicted_quantity * ingredient_qty
        
        # Add to the dictionary
        if ingredient in ingredient_quantities:
            ingredient_quantities[ingredient] += required_quantity
        else:
            ingredient_quantities[ingredient] = required_quantity

In [34]:
# Create a DataFrame from the ingredient quantities
ingredient_requirements_df = pd.DataFrame.from_dict(ingredient_quantities, orient='index', columns=['required_quantity'])
ingredient_requirements_df

,required_quantity
Barbecued Chicken,5554.579650
Red Peppers,11192.015949
Green Peppers,7683.637506
Tomatoes,29914.962419
Red Onions,44887.814993
...,...
Luganega Sausage,445.626231
Onions,222.813116
Soppressata Salami,5304.511970
Peperoncini verdi,917.818086


In [35]:
# Determine the last date in historical data
last_date = sales_pivot.index[-1]

# Generate date range for predictions
prediction_days = 7  # Number of days to predict
predicted_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=prediction_days, freq='D')

# Assign predicted dates to predictions DataFrame
predictions_df.index = predicted_dates
predictions_df

,The Barbecue Chicken Pizza,The Big Meat Pizza,The Brie Carre Pizza,The Calabrese Pizza,The California Chicken Pizza,The Chicken Alfredo Pizza,The Chicken Pesto Pizza,The Classic Deluxe Pizza,The Five Cheese Pizza,The Four Cheese Pizza,...,The Prosciutto and Arugula Pizza,The Sicilian Pizza,The Soppressata Pizza,The Southwest Chicken Pizza,The Spicy Italian Pizza,The Spinach Pesto Pizza,The Spinach Supreme Pizza,The Spinach and Feta Pizza,The Thai Chicken Pizza,The Vegetables + Vegetables Pizza
2016-01-01,8.160714,3.027896,0.191648,2.268178,6.507168,2.194045,3.389738,7.393368,3.948329,7.727451,...,4.544626,0.634973,1.745454,4.797908,2.419269,4.851607,2.553220,3.056631,5.351353,7.437561
2016-01-02,7.773062,7.377793,0.865479,3.374618,6.495245,3.052564,4.267490,7.561603,3.641112,11.749415,...,5.289952,0.803174,2.991464,4.849589,5.233762,5.148542,3.993360,2.522785,8.236196,7.408238
2016-01-03,7.283069,4.185966,1.005429,2.077624,6.494081,1.072046,2.997033,7.445395,3.195455,11.074673,...,2.813993,2.336787,2.004835,3.320830,5.127789,5.581758,1.529954,1.951653,6.408649,10.577717
2016-01-04,9.611882,3.542319,0.125743,1.223214,6.504170,2.943628,4.886369,8.677715,2.043000,7.688107,...,5.802684,-0.927236,2.840617,3.355027,2.500047,3.522347,3.018285,4.209406,3.745625,7.307964
2016-01-05,8.617341,5.322685,0.559086,3.652895,6.508639,2.639193,1.682049,6.206635,2.455170,8.659632,...,3.670766,-0.218570,2.980137,4.308192,3.009983,4.824944,1.017486,1.594424,4.836520,5.279084
2016-01-06,8.561071,5.052242,-0.380149,1.688557,7.011159,2.731303,2.341016,5.180927,2.372682,11.341155,...,3.160844,-2.229550,2.120870,6.361716,4.504393,4.707990,2.044443,1.660103,3.349258,7.517579
2016-01-07,11.710412,5.363211,1.587643,2.167527,6.015844,3.524026,3.658955,9.071744,4.949138,8.608578,...,5.283026,3.313974,2.998330,5.841193,7.798694,5.231970,1.557527,4.172336,3.871841,11.625989


In [36]:
# Create a purchase order DataFrame
purchase_order_df = ingredient_requirements_df.copy()

# Add a column for the unit of measure (assuming all ingredients are in grams)
purchase_order_df['unit'] = 'grams'

# Rename the columns for better readability
purchase_order_df = purchase_order_df.rename(columns={'required_quantity': 'quantity'})

# Add a header to the purchase order
print('Purchase Order:')
# print('----------------')

# Print the purchase order table
print(purchase_order_df.to_string())

Purchase Order:
                                quantity   unit
Barbecued Chicken            5554.579650  grams
Red Peppers                 11192.015949  grams
Green Peppers                7683.637506  grams
Tomatoes                    29914.962419  grams
Red Onions                  44887.814993  grams
Barbecue Sauce               1851.526550  grams
Bacon                       18995.910759  grams
Pepperoni                   22477.214206  grams
Italian Sausage               338.721110  grams
Chorizo Sausage              1693.605548  grams
Brie Carre Cheese             118.646399  grams
Prosciutto                    118.646399  grams
Caramelized Onions             63.278080  grams
Pears                          39.548800  grams
Thyme                          19.774400  grams
Garlic                      15418.184086  grams
?duja Salami                 1974.313619  grams
Pancetta                     2961.470428  grams
Friggitello Peppers           493.578405  grams
Chicken                 

In [37]:
# purchase_order_df.to_csv("purchase_order.txt")